# Weak-lensing galaxy shape catalogue validation

## Write catalogues

Contents
- Write catalogue FITS files

> **_NOTE:_** Before running this notebook, set kernel to `main_set.ipynb'

In [ ]:
import os

In [ ]:
from sp_validation.util import *
from sp_validation.cat import *
from sp_validation.basic import metacal

In [ ]:
R_shear_ind = {}
for sh in shapes:
    # Shear response per galaxy
    R_shear_ind[sh] = gal_metacal[sh].R_shear

### Write basic shape catalogue

In [ ]:
local_cal = False

if local_cal:
    for sh in shapes:
        write_shape_catalog(
            f'{output_shape_cat_base}_{sh}.fits',
            ra[sh],
            dec[sh],
            w[sh],
            mag=mag[sh],
            g=g_corr_mc[sh],
            g1_uncal=g_uncorr[sh][0],
            g2_uncal=g_uncorr[sh][1], 
            R_g11=R_shear_ind[sh][0, 0],
            R_g12=R_shear_ind[sh][0, 1],
            R_g21=R_shear_ind[sh][1, 0],
            R_g22=R_shear_ind[sh][1, 1],
            R=gal_metacal[sh].R,
            R_shear=R_shear[sh],
            R_select=gal_metacal[sh].R_selection,
            c=c[sh],
            c_err=c_err[sh],
            alpha_leakage=alpha_leak_mean[sh],
            add_cols=add_cols_data[sh],
        )
else:
    for sh in shapes:   
        write_shape_catalog(
            f'{output_shape_cat_base}_{sh}.fits',
            ra[sh],
            dec[sh],
            w[sh],
            mag=mag[sh],
            g=g_corr_mc[sh],
            g1_uncal=g_uncorr[sh][0],
            g2_uncal=g_uncorr[sh][1],
            R=gal_metacal[sh].R,
            R_shear=R_shear[sh],
            R_select=gal_metacal[sh].R_selection,
            c=c[sh],
            c_err=c_err[sh],
            alpha_leakage=alpha_leak_mean[sh],
            add_cols=add_cols_data[sh],
        )

### Write extended shape catalogue

In [ ]:
ext_cols = {}
for sh in shapes:
    if add_cols:
        ext_cols[sh] = add_cols_data[sh]
    else:
        ext_cols[sh] = {}

In [ ]:
# Optional: Create flag from external mask
if mask_external_path:
    for sh in shapes:
        m_extern = mask_overlap(ra[sh], dec[sh], tile_ID[sh], mask_external_path)

In [ ]:
for sh in shapes:

    # Additional columns:
    # {e1, e2, size}_PSF
    ext_cols[sh]['e1_PSF'] = dd[key_PSF_ell[sh]][:,0][m_gal[sh]][mask[sh]]
    ext_cols[sh]['e2_PSF'] = dd[key_PSF_ell[sh]][:,1][m_gal[sh]][mask[sh]]
    ext_cols[sh]['fwhm_PSF'] = size_to_fwhm[sh](dd[key_PSF_size[sh]][m_gal[sh]][mask[sh]])
    if mask_external_path:
        ext_cols[sh]['mask_extern'] = m_extern

    # Extended catalogue with SNR, individual R matrices, ext_cols
    write_shape_catalog(
        f'{output_shape_cat_base}_extended_{sh}.fits',
        ra[sh],
        dec[sh],
        w[sh],
        mag=mag[sh],
        snr=snr[sh],
        g=g_corr_mc[sh],
        g1_uncal=g_uncorr[sh][0],
        g2_uncal=g_uncorr[sh][1],
        R_g11=R_shear_ind[sh][0, 0],
        R_g12=R_shear_ind[sh][0, 1],
        R_g21=R_shear_ind[sh][1, 0],
        R_g22=R_shear_ind[sh][1, 1],       
        R=gal_metacal[sh].R,
        R_shear=R_shear[sh],
        R_select=gal_metacal[sh].R_selection,
        c=c[sh],
        c_err=c_err[sh],
        alpha_leakage=alpha_leak_mean[sh],
        add_cols=ext_cols[sh],
     )

### Write comprehensive shape catalogue (all objects, no cuts)

In [ ]:
# Add additional columns without cuts nor mask applied

ext_cols_pre_cal = {}

for sh in shapes:
    ext_cols_pre_cal[sh] = {}

    # Standard additional columns
    if add_cols:
        for key in add_cols:
            ext_cols_pre_cal[sh][key] = dd[key]

    # Pre-calibration columns
    if add_cols_pre_cal:
        for key in add_cols_pre_cal:
            ext_cols_pre_cal[sh][key] = dd[key]

    # Flag to cut duplicate objects in overlapping region with neighbouring tiles
    ext_cols_pre_cal[sh]["overlap"] = cut_overlap
    add_cols_pre_cal_format["overlap"] = "I"

    # Additional columns {e1, e2, size}_PSF
    ext_cols_pre_cal[sh]['e1_PSF'] = dd[key_PSF_ell[sh]][:,0]
    ext_cols_pre_cal[sh]['e2_PSF'] = dd[key_PSF_ell[sh]][:,1]
    ext_cols_pre_cal[sh]['fwhm_PSF'] = size_to_fwhm[sh](dd[key_PSF_size[sh]])

    _, _, iv_w = metacal.get_variance_ivweights(dd, sigma_eps_prior, mask=None, col_2d=True)

    mag = get_col(dd, "MAG_AUTO", None, None)
    snr = get_snr(sh, dd, None, None)
    g1_uncal = dd["NGMIX_ELL_NOSHEAR"][:, 0]
    g2_uncal = dd["NGMIX_ELL_NOSHEAR"][:, 1]
    
    # Comprehensive catalogue without cuts nor mask applied
    write_shape_catalog(
        f'{output_shape_cat_base}_comprehensive_{sh}.fits',
        ra["all"],
        dec["all"],
        iv_w,
        mag=mag,
        snr=snr,
        g1_uncal=g1_uncal,
        g2_uncal=g2_uncal,
        add_cols=ext_cols_pre_cal[sh],
        add_cols_format=add_cols_pre_cal_format,
     )

### Write galaxy (or random) position catalogue

In [ ]:
if len(shapes) == 0:
    print('writing random cat (hack)')
        
    ra = dd['RA'][cut_overlap]
    dec = dd['DEC'][cut_overlap]
    #tile_id_str = [f'{id:07.3f}' for id in dd['TILE_ID'][cut_overlap]]
    tile_id = dd['TILE_ID'][cut_overlap]
    write_galaxy_cat(f'{output_shape_cat_base}.fits', ra, dec, tile_id)

### Write PSF catalogue with multi-epoch shapes from shape measurement methods

In [ ]:
for sh in shapes:                                                               
                                                                                
    write_PSF_cat(                                                              
        f'{output_PSF_cat_base}_{sh}.fits',                                     
        ra_star[sh],                                                            
        dec_star[sh],                                                           
        g_star_psf[sh][0],                                                      
        g_star_psf[sh][1]                                                       
    ) 